<a href="https://colab.research.google.com/github/mdrinkard/Project-2/blob/main/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Installing Required Modules on Google
!pip install hvplot
!pip install tensorflow
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.3 MB/s eta 0:00:00


In [ ]:
### Importing those Modules
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [ ]:
### Uploading Dataset
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]


Saving SBA_CLEANED.csv to SBA_CLEANED.csv


In [ ]:
### Creating Dataframe
df = pd.read_csv('SBA_CLEANED.csv')
df

,State,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,MIS_Status,Industry,TermGroup,NewBankState,IsFranchise,NewCity,NewBank
0,IN,4,2.0,0,0,0,N,Y,0,Retail Trade,Below 3 months,OH,Y,EVANSVILLE,FIFTH THIRD BANK
1,IN,3,2.0,0,0,0,N,Y,0,Construction,Below 3 months,OH,Y,EVANSVILLE,FIFTH THIRD BANK
2,IN,4,1.0,0,0,0,N,Y,1,Retail Trade,Below 3 months,OH,Y,EVANSVILLE,FIFTH THIRD BANK
3,IN,11,1.0,0,0,0,N,N,0,Unknown Sector,Below 3 months,OH,Y,EVANSVILLE,FIFTH THIRD BANK
4,IN,7,2.0,0,7,2,N,N,1,Retail Trade,Below 3 months,OH,Y,EVANSVILLE,FIFTH THIRD BANK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885300,KY,3,2.0,0,0,0,N,Y,0,Construction,Below 3 months,KY,Y,OTHER,OTHER
885301,KY,2,2.0,0,0,0,N,Y,0,Unknown Sector,Below 3 months,KY,Y,OTHER,OTHER
885302,NE,15,2.0,0,0,0,N,N,1,Unknown Sector,3-6 months,IL,Y,OTHER,OTHER
885303,GA,2,2.0,0,0,0,N,Y,0,Unknown Sector,3-6 months,GA,Y,OTHER,OTHER


In [ ]:
def report(df):
    col = []
    d_type = []
    uniques = []
    n_uniques = []
    nan = []

    for i in df.columns:
        col.append(i)
        d_type.append(df[i].dtypes)
        uniques.append(df[i].unique()[:5])
        n_uniques.append(df[i].nunique())
        nan.append(df[i].isna().sum()/len(df)*100)

    return pd.DataFrame({'Column': col, 'd_type': d_type, 'unique_sample': uniques, 'n_uniques': n_uniques, 'nan%': nan})

report(df)

,Column,d_type,unique_sample,n_uniques,nan%
0,State,object,"[IN, FL, KY, NC, MI]",51,0.001468
1,NoEmp,int64,"[4, 3, 11, 7, 13]",597,0.000000
2,NewExist,float64,"[2.0, 1.0, nan, 0.0]",3,0.014684
3,CreateJob,int64,"[0, 7, 2, 8, 5]",245,0.000000
4,RetainedJob,int64,"[0, 7, 60, 20, 16]",356,0.000000
5,UrbanRural,int64,"[0, 2, 1]",3,0.000000
6,RevLineCr,object,"[N, Y]",2,0.000000
7,LowDoc,object,"[Y, N]",2,0.000000
8,MIS_Status,int64,"[0, 1]",2,0.000000
9,Industry,object,"[Retail Trade, Construction, Unknown Sector, M...",21,0.000000


In [ ]:
df_analysis = df.copy()
df_analysis = df_analysis.dropna()
df_analysis.drop(columns='NewCity', inplace=True)
display(df_analysis.isnull().sum())

State           0
NoEmp           0
NewExist        0
CreateJob       0
RetainedJob     0
UrbanRural      0
RevLineCr       0
LowDoc          0
MIS_Status      0
Industry        0
TermGroup       0
NewBankState    0
IsFranchise     0
NewBank         0
dtype: int64

In [ ]:
### We are ready to start working on the creating the skeleton for our neural network.
### As we have a number of categorical variables we will be using onehotencoder to transform the X variables.
### to make it applicable to the model.
display(df_analysis.dtypes)
categorical_variables = list(df_analysis.dtypes[df_analysis.dtypes == 'object'].index)
display(categorical_variables)


State            object
NoEmp             int64
NewExist        float64
CreateJob         int64
RetainedJob       int64
UrbanRural        int64
RevLineCr        object
LowDoc           object
MIS_Status        int64
Industry         object
TermGroup        object
NewBankState     object
IsFranchise      object
NewBank          object
dtype: object

['State',
 'RevLineCr',
 'LowDoc',
 'Industry',
 'TermGroup',
 'NewBankState',
 'IsFranchise',
 'NewBank']

In [ ]:
enc= OneHotEncoder(sparse_output=False)
enc

OneHotEncoder(sparse_output=False)

In [ ]:
encoded_data = enc.fit_transform(df_analysis[categorical_variables])
encoded_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names_out(categorical_variables)
)
encoded_df.shape

(885162, 163)

In [ ]:
display(encoded_df.isnull().sum())

State_AK                                  0
State_AL                                  0
State_AR                                  0
State_AZ                                  0
State_CA                                  0
                                         ..
NewBank_THE HUNTINGTON NATIONAL BANK      0
NewBank_U.S. BANK NATIONAL ASSOCIATION    0
NewBank_UMPQUA BANK                       0
NewBank_WELLS FARGO BANK NATL ASSOC       0
NewBank_ZIONS FIRST NATIONAL BANK         0
Length: 163, dtype: int64

In [ ]:
unscaled_df = pd.concat([encoded_df, df_analysis.drop(columns=categorical_variables)],axis = 1)
unscaled_df

,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,...,NewBank_U.S. BANK NATIONAL ASSOCIATION,NewBank_UMPQUA BANK,NewBank_WELLS FARGO BANK NATL ASSOC,NewBank_ZIONS FIRST NATIONAL BANK,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,MIS_Status
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,2.0,0.0,7.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,2.0,0.0,0.0,0.0,0.0
885301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0
885302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15.0,2.0,0.0,0.0,0.0,1.0
885303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0


In [ ]:
display(encoded_df.shape)
display(df_analysis.shape)
display(unscaled_df.shape)

(885162, 163)

(885162, 14)

(885304, 169)

In [ ]:
unscaled_df.dropna(inplace=True)
unscaled_df.isnull().sum()

State_AK       0
State_AL       0
State_AR       0
State_AZ       0
State_CA       0
              ..
NewExist       0
CreateJob      0
RetainedJob    0
UrbanRural     0
MIS_Status     0
Length: 169, dtype: int64

In [ ]:
y = unscaled_df['MIS_Status'].astype(int)
X = unscaled_df.drop(columns='MIS_Status')

In [ ]:
display(y.shape)
X.shape

(885020,)

(885020, 168)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

In [ ]:
### Now to Scale my Data
scaler = StandardScaler()

X_scaler= scaler.fit(X)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
### Time to Start creating the Neural Network.
number_input_features= len(X_train.iloc[0])
number_input_features

168

In [ ]:
# Defining Layers
number_output_neurons = 1
hidden_node_1 = 200
hidden_node_2 = 100
# hidden_node_3 = 50
# hidden_node_4 = 25
# hidden_node_5 = 25
nn=Sequential()

In [ ]:
nn.add(Dense(
    units=hidden_node_1,
    input_dim=number_input_features,
    activation='relu'
))

In [ ]:
nn.add(Dense(
    units=hidden_node_2,
    activation='relu'
))

In [ ]:
# nn.add(Dense(
#     units=hidden_node_3,
#     activation='relu'
# ))

In [ ]:
# nn.add(Dense(
#     units=hidden_node_4,
#     activation='relu'
# ))

In [ ]:
# nn.add(Dense(
#     units=hidden_node_5,
#     activation='relu'
# ))

In [ ]:
nn.add(Dense(
    units=number_output_neurons,
    activation='sigmoid'
))

In [ ]:
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 200)               33800     
                                                                 
 dense_20 (Dense)            (None, 100)               20100     
                                                                 
 dense_21 (Dense)            (None, 1)                 101       
                                                                 
Total params: 54001 (210.94 KB)
Trainable params: 54001 (210.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
### Compiling the Model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batchsize=500

In [ ]:
fit_model = nn.fit(X_train_scaled,y_train, epochs=10, batch_size=batchsize)

Epoch 1/10
1328/1328 [==============================] - 10s 7ms/step - loss: 0.4199 - accuracy: 0.8264
Epoch 2/10
1328/1328 [==============================] - 8s 6ms/step - loss: 0.4121 - accuracy: 0.8281
Epoch 3/10
1328/1328 [==============================] - 9s 7ms/step - loss: 0.4103 - accuracy: 0.8286
Epoch 4/10
1328/1328 [==============================] - 9s 7ms/step - loss: 0.4090 - accuracy: 0.8290
Epoch 5/10
1328/1328 [==============================] - 8s 6ms/step - loss: 0.4080 - accuracy: 0.8293
Epoch 6/10
1328/1328 [==============================] - 9s 7ms/step - loss: 0.4071 - accuracy: 0.8296
Epoch 7/10
1328/1328 [==============================] - 9s 7ms/step - loss: 0.4063 - accuracy: 0.8297
Epoch 8/10
1328/1328 [==============================] - 8s 6ms/step - loss: 0.4055 - accuracy: 0.8301
Epoch 9/10
1328/1328 [==============================] - 9s 7ms/step - loss: 0.4046 - accuracy: 0.8301
Epoch 10/10
1328/1328 [==============================] - 9s 7ms/step - loss: 0.40

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6915/6915 - 10s - loss: 0.4093 - accuracy: 0.8294 - 10s/epoch - 1ms/step
Loss: 0.40927278995513916, Accuracy: 0.8293688297271729


AttributeError: 'Sequential' object has no attribute 'hvplot'